In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_chloride_abnormal,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps
201220,125078,66690,0,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,1,1,7.45,7.42,7.45,1,1,0,1,0.4496715
215842,151232,11663,0,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,0,0,NA,NA,NA,NA,0,0,0,0.1577062
234312,164444,86645,1,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,1,1,7.49,7.49,7.49,1,0,0,0,0.3034738
289157,146726,10304,0,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,0,1,7.44,7.44,7.44,1,0,1,0,0.7020434
211964,160170,94534,0,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,0,0,NA,NA,NA,NA,0,0,0,0.3399883
230173,112553,31544,0,2140-01-30 20:39:25,2140-02-02 18:41:39,1,91.50000,1,NA,⋯,1,0,NA,NA,NA,NA,0,0,1,0.2497070


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
1.942281e-03,0.04772662,84.183688,79.166557,67.137454,59.7201585,83.089515,79.918203,64.906984,60.763406,age
1.416420e-02,0.34205147,50.822846,47.665687,NA,NA,49.415944,48.205909,NA,NA,female
6.091011e-02,0.03565708,1.801389,1.837418,0.738580,0.7696856,2.104114,2.159730,1.036885,1.040762,weight
1.705518e-15,0.08790062,8.505970,10.045054,7.451124,7.6816088,9.106113,9.436513,7.555664,7.638541,elix_score
1.378591e-15,0.12425987,19.632462,20.778975,5.788347,5.4462928,20.019478,20.236554,5.657332,5.425510,saps
7.477319e-27,0.02807578,5.305260,6.323865,3.618029,3.7963648,5.662584,5.868751,3.669404,3.697442,sofa


In [5]:
wpv <- table_one %>% filter(wtd_p_val < 0.05) %>% pull(var)
wpv

[1] "age"                        "weight"                    
 [3] "sofa"                       "vent"                      
 [5] "vaso"                       "sedative"                  
 [7] "icd_chf"                    "icd_afib"                  
 [9] "lab_chloride_first"         "lab_bnp_flag"              
[11] "lab_troponin_flag"          "lab_creatinine_kinase_flag"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + age + weight + sofa + vent + vaso + sedative + icd_chf + icd_afib + lab_chloride_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -3.084729   0.537148  -5.743 9.77e-09 ***
echo1                       -0.195067   0.071792  -2.717 0.006604 ** 
age                          0.027820   0.002504  11.110  < 2e-16 ***
weight                       0.144464   0.043410   3.328 0.000880 ***
sofa                         0.217710   0.011515  18.907  < 2e-16 ***
vent1                        0.472870   0.110494   4.280 1.90e-05 ***
vaso1                       -0.028651   0.084757  -0.338 0.735347    
sedative1                   -0.396807   0.110515  -3.591 0.000333 ***
icd_chf1                    -0.153666   0.083352  -1.844 0.065296 .  
icd_afib1                    0.101197   0.082003   1.234 0.217228    
lab_chloride_first          -0.013394  

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.04574245,0.01596239,0.1310814
echo1,0.82277932,0.71478258,0.9470933
age,1.02821095,1.02317693,1.0332697
weight,1.15542043,1.06117999,1.2580301
sofa,1.24322665,1.21548351,1.2716030
vent1,1.60459234,1.29214760,1.9925871
vaso1,0.97175546,0.82302232,1.1473670
sedative1,0.67246353,0.54149931,0.8351021
icd_chf1,0.85755869,0.72830640,1.0097493
icd_afib1,1.10649486,0.94221077,1.2994236
